In [1]:
import geopandas as gpd
import pandas as pd

### Load Data

Data source: 
1. tax data: https://www.sfassessor.org/news-information/property-data-0
2. historic 

In [5]:
tax = pd.read_excel('./2023.7.18_SF_ASR_Secured_Roll_Data_2022-2023.xlsx')

In [4]:
historic = gpd.read_file('./Historic Districts/geo_export_a0505c4f-e263-4190-90bd-fe371826c801.shp')

In [28]:
historic = historic[['name_1', 'descriptio', 'geometry']]

In [75]:
parcels = gpd.read_file('./parcels/geo_export_ccf5bc9c-86d7-4a66-85d1-6e4ae4490c28.shp')

### Clean Assesor Parcel Number (aka blklot)

In [19]:
tax['blklot'] = tax.RP1PRCLID.str.replace(' ', '')

### Create geospatial version of tax assessor data so I can do a geospatial merge with the historic districts dataset

In [52]:
# better to merge on blklot than mapblklot
tax['blklot'].isin(parcels.blklot).mean()

0.9990404205132568

In [25]:
df = pd.merge(parcels[['blklot', 'geometry']], tax)

In [33]:
df.shape

(211348, 34)

In [34]:
# geospatial merge
full_df = gpd.sjoin(df, historic, predicate='within', rsuffix='historic', how='left')

In [36]:
full_df['is_historic'] = full_df['index_historic'].notna()

In [38]:
# join creates ~750 new rows bc these parcels are in multiple historic districts
full_df.shape[0] - df.shape[0]

746

In [41]:
# so, I drop extra matches
full_df = full_df.drop_duplicates(['PROPLOC', 'blklot', 'geometry'])

### How many parcels have buildings that are 100+ years old?

In [72]:
# When YRBLT is 0 or predates the country's founding, I interpret that as missing data
(full_df.YRBLT < 1776).mean().round(3)

0.056

In [68]:
((full_df['YRBLT'] > 1776) & (full_df['YRBLT'] <= 1923)).mean().round(3)

0.314

### How many parcels are in historic districts?

In [66]:
full_df['is_historic'].mean().round(3)

0.098

### How many parcels are in historic districts or have buildings that are 100+ years old?


In [74]:
# This is where the 41% figure came from
(full_df['is_historic'] | (full_df['YRBLT'] <= 1923)).mean().round(4)

0.4148

In [69]:
# Actual number is 36%
(full_df['is_historic'] | ((full_df['YRBLT'] > 1776) & (full_df['YRBLT'] <= 1923))).mean().round(3)

0.362